# Dataset Pre-Processing
This notebook contains the dataset pre-processing steps and considerations



In [ ]:
import os
import urllib

from google.colab import drive, files
from getpass import getpass

from google.colab import drive



In [ ]:
ROOT = '/content/drive'
GOOGLE_DRIVE_PATH = 'My Drive/Colab Notebooks/recommender/w266-final'
PROJECT_PATH = os.path.join(ROOT, GOOGLE_DRIVE_PATH)

drive.mount(ROOT)

%cd {PROJECT_PATH}


Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/recommender/w266-final


In [ ]:
import re
import pandas as pd
import numpy as np
import itertools
import pickle

from commons.store import PickleStore, NpyStore
from tqdm import tqdm
from IPython.core.display import HTML

from importlib import reload

%load_ext autoreload
%autoreload 2

## 1. Load pre-filtered dataset
Here our dataset(s) have been pre-filtered where only reviews with tokens length between 25 and 65 (exclusive) are selected. Shorter reviews tend to be less meanigful, while longers ones tend to contain garbages.

In [ ]:
amazon = False
if amazon:
  input_pkl = '../dataset/reviews_25-65_tokens_with_user-item_counts.pkl'
  features = ['reviewerID', 'asin', 'overall', 'reviewText']

else:
  input_pkl = '../dataset/yelp_25_64_tokens_10_reviews_all.pkl'
  features = ['reviewerID', 'asin', 'stars', 'text']

reviews = pd.read_pickle(input_pkl)[features]

if not amazon:
  reviews.rename(columns={'stars': 'overall', 'text': 'reviewText'}, inplace=True)


display(HTML(reviews.head(2).to_html()))

,reviewerID,asin,overall,reviewText
17,JHXQEayrDHOWGexs0dCviA,KXCXaF5qimmtKKqnPc_LQA,1,Great coffee and pastries. Baristas are excellent. All the other staff are sooooo lost and vacant.
18,DECuRZwkUw8ELQZfNGef2Q,zmZ3HkVCeZPBefJJxzdJ7A,4,"Nothing special but good enough. I like another one much better in Dorchester, but hardly get to that area, so Le's is my goto place for Pho"


## 2. Further filtering on reviews
We further pre-process the reviews such as unnecessary punctuations removal as we are only interested on the overall semantic meaning of reviews.

In [ ]:
def translate(text):
    """apply filters on given text"""
    
    filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    
    # mask URL
    text = re.sub(r'(\w+:\/{2}|w{3}.*\.)[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s\/]*))*', 
                  '', text, flags=re.IGNORECASE)
    
    table = str.maketrans('', '', filters)
    words = str(text.lower())
    
    return words.translate(table)

def clean_seq(dataframe):
    """clean up reviews sequence"""
    return dataframe.apply(
            lambda row: translate(row.reviewText), axis=1)

def filter_n_reviews(dataframe, n=5):
    """filter rows with n and more number of reviews for each reviewers and items"""
    dataframe = dataframe[dataframe['reviewText'].groupby(dataframe['reviewerID']).transform('size') >= n]
    dataframe = dataframe[dataframe['reviewText'].groupby(dataframe['asin']).transform('size') >= n]
    
    return dataframe

In [ ]:
%time reviews['reviewText'] = clean_seq(reviews)

display(HTML(reviews.head(2).to_html()))

CPU times: user 15 s, sys: 130 ms, total: 15.1 s
Wall time: 15.1 s


,reviewerID,asin,overall,reviewText
17,JHXQEayrDHOWGexs0dCviA,KXCXaF5qimmtKKqnPc_LQA,1,great coffee and pastries baristas are excellent all the other staff are sooooo lost and vacant
18,DECuRZwkUw8ELQZfNGef2Q,zmZ3HkVCeZPBefJJxzdJ7A,4,nothing special but good enough i like another one much better in dorchester but hardly get to that area so le's is my goto place for pho


We group reviews on user and item. In other words, all the reviews written by a user are grouped for that user. Similarly, all the reviews written for an item are grouped under that item. The caveat is we remove a review written by that user and item so the model will predict the rating for that user and item based on other reviews written by that user and other items written for that item. This essentially solves the cold-start problem in recommendation system.

We then further filter the reviews that have *n* number of reviews on each groups of user and item as we want to capture the user interactions with items based on posted reviews.

In [ ]:
reviews['reviewerID'] = reviews['reviewerID'].astype('str')
reviewer_sorted = reviews.sort_values('reviewerID')

reviews['asin'] = reviews['asin'].astype('str')
item_sorted = reviews.sort_values('asin')

In [ ]:
def merge_reviews(arr):
    # find the start and end indexes of the sorted user review data
    user_review_start_index = reviewer_sorted['reviewerID'].searchsorted(arr[0],'left')
    user_review_end_index = reviewer_sorted['reviewerID'].searchsorted(arr[0],'right')
    # we need at least 2 rows because the current combination is filtered out
    if user_review_end_index - user_review_start_index < 2:
        return None
    # use the index range to get the rows with tokenized reviews by this user
    user_reviews = reviewer_sorted.iloc[user_review_start_index:user_review_end_index+1]
    # get all the user reviews except the one for the current item
    user_review_list = np.take(user_reviews['reviewText'].to_numpy(), 
                               np.where(user_reviews['asin'].to_numpy() != arr[1]))[0]
    
    # find the start and end indexes of the sorted item review data
    item_review_start_index = item_sorted['asin'].searchsorted(arr[1],'left')
    item_review_end_index = item_sorted['asin'].searchsorted(arr[1],'right')
    # we need at least 2 rows because the current combination is filtered out
    if item_review_end_index - item_review_start_index < 2:
        return None
    # use the index range to get the rows with tokenized reviews for this item
    item_reviews = item_sorted.iloc[item_review_start_index:item_review_end_index+1]
    item_review_list = np.take(item_reviews['reviewText'].to_numpy(), 
                               np.where(item_reviews['reviewerID'].to_numpy() != arr[0]))[0]
    
    return (arr[0], arr[1], arr[2], user_review_list, item_review_list)

In [ ]:
total = len(reviews)
step = 500
n = total // step
iterator = (itertools.count(start = 0, step = step))

grouped_reviews, end = [], 0
with tqdm(total=n) as pb:
    for i in (next(iterator) for _ in range(n+1)):
        end = i + step
        if end > total:
            end = total
        
        grouped_reviews.extend([merge_reviews(review) for review in reviews.iloc[i:end].to_numpy()])
        pb.update(1)

713it [03:33,  3.34it/s]                         


In [ ]:
grouped_reviews_df =  pd.DataFrame.from_records(grouped_reviews, 
                                       columns=['reviewerID', 'asin', 'overall', 'userReviews', 'itemReviews'])
display(HTML(grouped_reviews_df.head(1).to_html()))

In [ ]:
grouped_reviews_df.dropna(inplace=True)
grouped_reviews_df['userCount'] = grouped_reviews_df.userReviews.map(len)
grouped_reviews_df['itemCount'] = grouped_reviews_df.itemReviews.map(len)

In [ ]:
n = 7
columns = ['reviewerID', 'asin', 'overall', 'userReviews', 'itemReviews']
grouped_reviews_df = grouped_reviews_df[grouped_reviews_df.userCount >= n]
grouped_reviews_df = grouped_reviews_df[grouped_reviews_df.itemCount >= n]
grouped_reviews_df = grouped_reviews_df[columns]

## 3. Pickle the ready dataset

In [ ]:
output_path = '../dataset'
if amazon:
  output_pkl = os.path.join(output_path, '25-65_tokens_grouped_Movies_and_TV_v2.pkl')
else:
  output_pkl = os.path.join(output_path, '25-65_tokens_grouped_yelp.pkl')

# make dir if not exists
if not os.path.exists(output_path):
    os.makedirs(output_path)

# remove if exists
if os.path.exists(output_pkl):
    os.remove(output_pkl)

pkl_store = PickleStore(output_pkl)
pkl_store.write(grouped_reviews_df.to_numpy())

Saving ../dataset/25-65_tokens_grouped_yelp.pkl ...
	... 272296 records
Done!!
